# AutoAI  Webサービスの呼び出し
このnotebookはIBMCloud上のWatson Studioのnotebookで動くように書かれています。

## 0. 事前準備
以下の２つを事前に実施お願いします。既に実施済みの場合はSetupに進んでください。

### 1 [Watson Studio上のnotebookからIBM Cloud Object Storage(ICOS)へのFileの読み書き - project-libを使う -](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40)の手順が前準備として必要です。

以下を実施お願いします:
#### 1. 別のブラウザーのタブで [1. Projectを開く](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40)　を実施。開くプロジェクトはこのNotenookを作成したプロジェクトです。
#### 2. 1で開いたプロジェクトのタブで[2. 前準備 Access tokenの作成](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#2-%E5%89%8D%E6%BA%96%E5%82%99-access-token%E3%81%AE%E4%BD%9C%E6%88%90) を実施
#### 3. このNotebookで　[5: Project tokenのInsert](https://qiita.com/nishikyon/items/1bed62a2a98b0c970f40#5-project-token%E3%81%AEinsert)を実施してください。最後に作成された一番上のセルの実行を忘れないようにしてください。

### 2. UCI Iris.csvのプロジェクトへの読み込み
[Watson StudioのGalleryからデータをプロジェクトにロードする](https://qiita.com/nishikyon/items/4d97db14f42ab7c41a3e)を実施してください。

### 3. AutoAIで UCI Iris.csvの予測モデルを作り、`Iris Deployment` という名前でDeployする
AutoAIは[AutoAI の概説](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-overview.html) を参照してください。


## 1. セットアップ

以下のセットアップ作業を行なってください
 - 　古いクラインアント`watson-machine-learning-client`　のアンインストール
 - `watson-machine-learning-client-V4` のインストール

In [ ]:
!pip uninstall watson-machine-learning-client -y

In [ ]:
!pip install -U watson-machine-learning-client-V4

## 2. UCI Iris.csvのデータフレームへの読み込み
ファイルの読み書きの詳細を知りたい場合は　[Python 用の project-lib の使用](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/project-lib-python.html)を参照してください。

`UCI Iris.csv`　の１行目のデータをtest_iris_dfに入れます

In [ ]:
csv_file_name = 'UCI Iris.csv'
csv_file = project.get_file(csv_file_name)
csv_file.seek(0)

import pandas as pd
iris_df = pd.read_csv(csv_file)

#最初の5行の表示
display(iris_df.head())

#テストデータとして１行目のDataframeを抜き出す
test_iris_df = iris_df.drop(columns='CLASS').iloc[0:1]
display(test_iris_df)


# 2. Watson Machine Learningの資格情報の取得 <a name="credential"></a>

[ダッシュボード](https://cloud.ibm.com/)→[リソースの表示](https://cloud.ibm.com/resources)の`Services`の下からWatson Machine　Learningサービスの名前をクリックします。

詳細が作成されたら、画面左側の`サービス資格情報`リンクをクリックします。`新規資格情報`ボタンをクリックし、資格情報に名前を付けて、追加をクリックします。新しいクレデンシャルは、`資格情報の表示`ボタンをクリックしてアクセスできます。表示された資格情報をコピーして下のセルに貼り付けます。

In [ ]:
#自分のWatson Machine Learningの資格情報をペーストしてください。
wml_credentials = {
  "apikey": "<ここにコピペ>",
  "instance_id": "<ここにコピペ>",
  "url": "<ここにコピペ>"
}

## 3. Deployment Nameのセット
`Iris Deployment`という名前で作成した場合は、そのまま実行してください。
違う名前でDeploymentを作成した場合は、`Iris Deployment`部分を自分のDeployment　Nameに修正してから実行してください。

In [ ]:
DEPLOYMENT_NAME = 'Iris Deployment'

## 4. DeplymenyのIDの取得
以下を実行してください。

In [ ]:
from watson_machine_learning_client.deployment import WebService


service = WebService(wml_credentials)
service_list_df = service.list()
display(service_list_df)
for index,item in service_list_df.iterrows():
    if item['name'] == DEPLOYMENT_NAME:
        deployment_id = item['id']
        break
print ('The Id is' +deployment_id)

## 5. Webサービスを呼び出して予測結果の取得
 `UCI Iris.csv`　の１行目のデータの予測をします。以下を実行してください。

In [ ]:
service.get( deployment_id) 
predictions = service.score(payload=test_iris_df)
predictions

`value`に`Iris-setosa`が予測されたと思います。